---

Las **imágenes** a utilizar en este cuaderno son:

- texto.jpg
- sudoku.jpg
- digito.jpg
- senial_prohibido_90.jpg

---

**Montamos el Drive** de nuestro repositorio de Google para tener acceso a las imágenes que empleamos en este cuaderno:

---


In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount = True)

---

# **Tesseract OCR**

[Tesseract](https://pypi.org/project/pytesseract/) es una biblioteca de funciones en Python para el reconocimiento de texto (caracteres y digitos) en imágenes digitales (fue desarrollada originalmente por Hewlett-Packard y su desarrollo fue adquirido después por Google; por eso ahora se conoce como “Google Tesseract OCR”.

Para aprender más usos y funcionalidades de Tesseract:
[OCR using Pytesseract and OpenCV](https://nanonets.com/blog/ocr-with-tesseract/)

Necesitamos instalar Tesseract para después poder importarlo en nuestro código.

Lo instalamos haciendo uso de '!' para hacer llamadas al sistema (en este caso al comando sudo para poder llamar al comando apt para instalar Tesseract ya que el comando apt requiere permisos de root):

---


In [ ]:
!sudo apt install tesseract-ocr
!pip install pytesseract
import pytesseract as pyt

---

Ahora comprobamos los idiomas soportados por la instalación que hemos realizado

---


In [ ]:
print(f'\nLos idiomas soportados por Tesseract son: {pyt.get_languages()}')

---

Si no soporta el idioma español podemos instalarlo por separado y volvemos a comprobar si ya tiene soporte para español:

---


In [ ]:
!sudo apt install tesseract-ocr-spa
print(f'\nLos idiomas soportados por Tesseract son: {pyt.get_languages()}')

---

Importamos el resto de librerias que vamos a necesitar en este cuaderno:

---


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

---

Configuración:

- OCR Engine Modes:

  0    Legacy engine only.

  1    Neural nets LSTM engine only.

  2    Legacy + LSTM engines.

  3    Default, based on what is available.

- Page Segmentation Modes:

  0    Orientation and script detection (OSD) only.

  1    Automatic page segmentation with OSD.

  2    Automatic page segmentation, but no OSD, or OCR.

  3    Fully automatic page segmentation, but no OSD. (Default)

  4    Assume a single column of text of variable sizes.

  5    Assume a single uniform block of vertically aligned text.

  6    Assume a single uniform block of text.

  7    Treat the image as a single text line.

  8    Treat the image as a single word.

  9    Treat the image as a single word in a circle.

 10    Treat the image as a single character.

 11    Sparse text. Find as much text as possible in no particular order.

 12    Sparse text with OSD.

 13    Raw line. Treat the image as a single text line bypassing hacks that are Tesseract-specific.

OCR Engine modes: (see https://github.com/tesseract-ocr/tesseract/wiki#linux)

---


## **Ejemplo 1**: Abrimos una imagen con sólo texto y lo probamos:

---


In [ ]:
path = "/content/drive/MyDrive/Curso_24_25/imas/"
nombre_imagen = 'texto.jpg'
path += nombre_imagen

imgGris = cv2.imread(path, cv2.IMREAD_GRAYSCALE)

_, binary = cv2.threshold(imgGris, 127, 255, cv2.THRESH_BINARY)

print('Imagen Original')
#cv2_imshow(imgGris)
print('Imagen Binaria')
#cv2_imshow(binary)

# motor ocr por defecto (--oem 3) y un único bloque de texto (--psm 6)
custom_config = r'--oem 3 --psm 6'

# en el resultado obtenido de la imagen original le añadimos el
# parámetro lang = spa para que reconozca en el idioma español
print('\nEl texto reconocido en la imagen original es: ')
print(pyt.image_to_string(imgGris, lang = 'spa', config = custom_config))

# en el resultado obtenido de la imagen binarizada dejarmos el idioma por defecto
print('\nEl texto reconocido en la imagen binaria es: ')
print(pyt.image_to_string(binary, config=custom_config))


---

## **Ejemplo 2**: Abrimos la imagen sudoku.jpg y lo probamos:

---


In [ ]:
path = "/content/drive/MyDrive/Curso_24_25/imas/"
nombre_imagen = 'sudoku.jpg'
path += nombre_imagen

imgBGR = cv2.imread(path, cv2.IMREAD_COLOR)
imgGris = cv2.imread(path, cv2.IMREAD_GRAYSCALE)

print('Imagen Original')
cv2_imshow(imgBGR)

_, binary = cv2.threshold(imgGris, 60, 255, cv2.THRESH_BINARY)

# motor ocr por defecto (--oem 3), un único bloque de texto (--psm 6)
# y sólo dígitos (outputbase digits)
custom_config = r'--oem 3 --psm 6 outputbase digits'

# obtenemos los dígitos presentes en la imagen con Tesseract
digitos = pyt.image_to_string(binary, config=custom_config)

# mostramos el resultado
print(f'\nLos dígitos reconocido en la imagen original son: {digitos}')


---

## **Ejemplo 3**: Ahora abrimos la captura de una única celda del sudoku y lo probamos:

---


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pytesseract as pyt
from google.colab.patches import cv2_imshow

path = "/content/drive/MyDrive/Curso_24_25/imas/"
nombre_imagen = 'digito.jpg'
path += nombre_imagen

imgBGR = cv2.imread(path, cv2.IMREAD_COLOR)
imgRGB = cv2.cvtColor(imgBGR, cv2.COLOR_BGR2RGB)
imgGris = cv2.imread(path, cv2.IMREAD_GRAYSCALE)

_, binary = cv2.threshold(imgGris, 80, 255, cv2.THRESH_BINARY)
_, binary_inv = cv2.threshold(imgGris, 80, 255, cv2.THRESH_BINARY_INV)
_, trunc = cv2.threshold(imgGris, 80, 255, cv2.THRESH_TRUNC)
_, tozero = cv2.threshold(imgGris, 80, 255, cv2.THRESH_TOZERO)
_, tozero_inv = cv2.threshold(imgGris, 80, 255, cv2.THRESH_TOZERO_INV)

titulos = ['Imagen Original', 'Imagen Binaria',
	'Imagen Binaria Inversa', 'Imagen Truncada',
  'Imagen Tozero', 'Imagen Tozero Inversa']

imagenes = [imgGris, binary, binary_inv, trunc, tozero, tozero_inv]

for i in range(len(imagenes)):
	plt.subplot(3, 3, i+1), plt.imshow(imagenes[i], 'gray')
	plt.title(titulos[i])
	plt.xticks([]), plt.yticks([])

plt.show()

# motor ocr por defecto (--oem 3), un único carácter (--psm 10)
# y sólo dígitos (outputbase digits)
custom_config = r'--oem 3 --psm 10 outputbase digits'
print(f'\nEl dígito reconocido en la imagen original es: {pyt.image_to_string(imgGris, config=custom_config)}')
print(f'El dígito reconocido en la imagen binaria es: {pyt.image_to_string(binary, config=custom_config)}')


---

## **Ejemplo 4**: Reconocer valores numéricos en una imagen de una señal de tráfico:

---


In [ ]:
path = "/content/drive/MyDrive/Curso_24_25/imas/"
nombre_imagen = 'senial_prohibido_90.jpg'
path += nombre_imagen

imgBGR = cv2.imread(path, cv2.IMREAD_COLOR)
imgGris = cv2.imread(path, cv2.IMREAD_GRAYSCALE)

print('Imagen Original: \n')
cv2_imshow(imgBGR)

_, binary = cv2.threshold(imgGris, 60, 255, cv2.THRESH_BINARY)

# configuramos pytesseract para que detecte sólo dígitos
custom_config = r'--oem 3 --psm 6 outputbase digits'

# otra forma de hacerlo consiste en especificar los caracteres a reconocer
# custom_config = r'--oem 3 --psm 6 tessedit_char_whitelist=0123456789'

# obtenemos los dígitos presentes en la imagen con Tesseract
digitos = pyt.image_to_string(binary, config=custom_config)

# mostramos el resultado
print(f'\nLos dígitos reconocidos en la imagen original son: {digitos}')
print(f'Por lo tanto, se trata de una señal de prohibido circular a más de {int(digitos)} km/h')


---

# **EJERCICIO 1**: Reconocer texto en carteles informátivos de tráfico

- Probar el funcionamiento del OCR para distintos carteles informátivos de tráfico

- Probar lo mismo con otras librerías OCR en Python (EasyOCR, docTR, Keras-OCR, Aspose.OCR, ...)

  https://www.analyticsvidhya.com/blog/2024/04/ocr-libraries-in-python/

  https://blog.aspose.com/es/ocr/python-ocr-library/

---


